### https://trello.com/c/xjAkL36o/54-merge-manual-raydium-and-pancakeswap-into-yield-report

In [10]:
import requests
import pandas as pd
import pymysql
import numpy as np
from datetime import datetime
import traceback2


stable_coins = "USDT, USDC, DAI, BUSD, SUSD, UST, TUSD".split(",")

In [9]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()


In [19]:
pancakswap_df = pd.read_csv('PancakeSwap_manual.csv')
raydium_df = pd.read_csv('Raydium_manual.csv')
pancakswap_df['includes_stablecoin'] = (pancakswap_df['Pair_1'].isin(stable_coins)) | (pancakswap_df['Pair_2'].isin(stable_coins))
raydium_df['includes_stablecoin'] = (raydium_df['Pair_1'].isin(stable_coins)) | (raydium_df['Pair_2'].isin(stable_coins))
pancakswap_df['Date'] = pd.to_datetime(pancakswap_df['Date'])
raydium_df['Date'] = pd.to_datetime(raydium_df['Date'])


In [20]:
raydium_df

,Date,Pair_1,Pair_2,Where,Farm_payment,Pool_rate,includes_stablecoin
0,2021-05-03,RAY,USDT,Raydium,RAY,190.37,True
1,2021-05-03,RAY,USDC,Raydium,RAY,176.53,False
2,2021-05-03,RAY,SRM,Raydium,RAY,116.91,False
3,2021-05-03,RAY,SOL,Raydium,RAY,88.81,False
4,2021-05-03,RAY,ETH,Raydium,RAY,121.54,False
5,2021-05-03,FIDA,RAY,Raydium,FIDA-RAY,119.89,False
6,2021-05-03,OXY,RAY,Raydium,OXY-RAY,98.18,False
7,2021-05-03,MAPS,RAY,Raydium,MAPS-RAY,114.11,False
8,2021-05-03,KIN,RAY,Raydium,KIN-RAY,185.63,False
9,2021-05-03,COPE,USDC,Raydium,COPE,548.96,False


In [35]:
insert_raydium_data_df(raydium_df)
insert_pancakswap_data_df(pancakswap_df)

2021-05-03 00:00:00 RAY USDT 190.37 RAY Raydium True
2021-05-03 00:00:00 RAY USDC 176.53 RAY Raydium False
2021-05-03 00:00:00 RAY SRM 116.91 RAY Raydium False
2021-05-03 00:00:00 RAY SOL 88.81 RAY Raydium False
2021-05-03 00:00:00 RAY ETH 121.54 RAY Raydium False
2021-05-03 00:00:00 FIDA RAY 119.89 FIDA-RAY Raydium False
2021-05-03 00:00:00 OXY RAY 98.18 OXY-RAY Raydium False
2021-05-03 00:00:00 MAPS RAY 114.11 MAPS-RAY Raydium False
2021-05-03 00:00:00 KIN RAY 185.63 KIN-RAY Raydium False
2021-05-03 00:00:00 COPE USDC 548.96 COPE Raydium False
2021-05-03 00:00:00 STEP USDC 1410.22 STEP Raydium False
2021-05-03 00:00:00 MEDIA USDC 417.02 MEDIA Raydium False
2021-05-03 00:00:00 CAKE BNB 74.33 1718847 CAKE PancakeSwap False
2021-05-03 00:00:00 BUSD BNB 25.61 997816 CAKE PancakeSwap False
2021-05-03 00:00:00 ETH BNB 12.04 529096265 CAKE PancakeSwap False
2021-05-03 00:00:00 USDT BUSD 13.49 472049664 CAKE PancakeSwap True
2021-05-03 00:00:00 BTCB BNB 15.13 421808142 CAKE PancakeSwap False

In [34]:

def insert_pancakswap_data_df(df):
    df = df.astype(object).where(pd.notnull(df), None)
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        dt = row["Date"].timestamp()
        try:
            pair2 = row["Pair_2"] if row["Pair_2"] else ' '
            print(row["Date"], row["Pair_1"], pair2, row["Pool_rate"], row["pool_tvl"], row["Farm_payment"], row["Where"], row["includes_stablecoin"])
            sql = "INSERT INTO `pancakswap_farm_data` (`timestamp_dt`, `pair1`,`pair2`,`pool_rate`,`pool_size`,`farm_payment`,`where`,`includes_stablecoin`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (dt, row["Pair_1"], pair2, row["Pool_rate"], row["pool_tvl"], row["Farm_payment"], row["Where"], row["includes_stablecoin"]))
        except Exception as e:
            print(e)
    connection.commit()
    cursor.close()
    connection.close()


def insert_raydium_data_df(df):
    df = df.astype(object).where(pd.notnull(df), None)
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        dt = row["Date"].timestamp()
        try:
            pair2 = row["Pair_2"] if row["Pair_2"] else ' '
            print(row["Date"], row["Pair_1"], pair2, row["Pool_rate"], row["Farm_payment"], row["Where"], row["includes_stablecoin"])
            sql = "INSERT INTO `raydium_farm_data` (`timestamp_dt`, `pair1`,`pair2`,`pool_rate`, `farm_payment`,`where`,`includes_stablecoin`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (dt, row["Pair_1"], pair2, row["Pool_rate"], row["Farm_payment"], row["Where"], row["includes_stablecoin"]))
        except Exception as e:
            print(e)
    connection.commit()
    cursor.close()
    connection.close()

In [24]:
pancakswap_df

,Date,Pair_1,Pair_2,Where,Farm_payment,Pool_rate,pool_tvl,includes_stablecoin
0,2021-05-03,CAKE,BNB,PancakeSwap,CAKE,74.33,1718847,False
1,2021-05-03,BUSD,BNB,PancakeSwap,CAKE,25.61,997816,False
2,2021-05-03,ETH,BNB,PancakeSwap,CAKE,12.04,529096265,False
3,2021-05-03,USDT,BUSD,PancakeSwap,CAKE,13.49,472049664,True
4,2021-05-03,BTCB,BNB,PancakeSwap,CAKE,15.13,421808142,False
5,2021-05-03,USDT,BNB,PancakeSwap,CAKE,18.78,338604357,True
6,2021-05-03,BUNNY,BNB,PancakeSwap,CAKE,26.66,118972005,False
7,2021-05-03,USDC,BUSD,PancakeSwap,CAKE,16.59,95709301,False
8,2021-05-03,UNI,BNB,PancakeSwap,CAKE,33.59,94300563,False
9,2021-05-03,BTCB,BUSD,PancakeSwap,CAKE,35.05,90762920,False
